In [25]:
import pandas as pd
df3 = pd.read_csv('kleague_final3.csv',encoding = 'cp949')
df3.columns.to_list()

['H/A',
 '결과',
 'Draw',
 'year',
 'Team_win',
 'Opponent_win',
 'Team_득점_avg',
 'Team_vs_Opponent_득점_avg',
 'Team_슈팅_avg',
 'Team_vs_Opponent_슈팅_avg',
 'Team_유효 슈팅_avg',
 'Team_vs_Opponent_유효 슈팅_avg',
 'Team_PA내 슈팅_avg',
 'Team_vs_Opponent_PA내 슈팅_avg',
 'Team_드리블 시도_avg',
 'Team_vs_Opponent_드리블 시도_avg',
 'Team_드리블 성공_avg',
 'Team_vs_Opponent_드리블 성공_avg',
 'Team_드리블 성공%_avg',
 'Team_vs_Opponent_드리블 성공%_avg',
 'Team_패스 시도_avg',
 'Team_vs_Opponent_패스 시도_avg',
 'Team_패스 성공_avg',
 'Team_vs_Opponent_패스 성공_avg',
 'Team_패스 성공%_avg',
 'Team_vs_Opponent_패스 성공%_avg',
 'Team_키패스_avg',
 'Team_vs_Opponent_키패스_avg',
 'Team_공격진영 패스 시도_avg',
 'Team_vs_Opponent_공격진영 패스 시도_avg',
 'Team_공격진영 패스 성공_avg',
 'Team_vs_Opponent_공격진영 패스 성공_avg',
 'Team_공격진영 패스 성공%_avg',
 'Team_vs_Opponent_공격진영 패스 성공%_avg',
 'Team_수비진영 패스 시도_avg',
 'Team_vs_Opponent_수비진영 패스 시도_avg',
 'Team_수비진영 성공_avg',
 'Team_vs_Opponent_수비진영 성공_avg',
 'Team_수비진영 성공%_avg',
 'Team_vs_Opponent_수비진영 성공%_avg',
 'Team_롱패스 시도_avg',
 'Team_vs_Opponent_

In [41]:
train_df = df3[df3['year'] != 6]
test_df = df3[df3['year'] == 6]

# 입력(features)와 출력(label) 분리
X_train = train_df.drop('결과', axis=1)
y_train = train_df['결과']
X_test = test_df.drop('결과', axis=1)
y_test = test_df['결과']

Team columns: ['Team_강원', 'Team_경남', 'Team_광주', 'Team_대구', 'Team_대전', 'Team_부산', 'Team_상주', 'Team_서울', 'Team_성남', 'Team_수원', 'Team_수원FC', 'Team_울산', 'Team_인천', 'Team_전남', 'Team_전북', 'Team_제주', 'Team_포항']


In [42]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SimpleRNN
from tensorflow.keras.utils import to_categorical

# 데이터 전처리
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Random Forest 모델로 변수 중요도 계산
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
feature_importances = rf_model.feature_importances_

# 변수 중요도를 데이터프레임으로 변환
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# 중요도 순으로 정렬
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("Feature importances sorted by Random Forest:")
print(importance_df)

# 상위 n개의 중요한 변수 선택 (예를 들어 상위 10개)
n = 10
top_features = importance_df.head(n)['Feature'].values

# 선택된 변수로 새로운 데이터셋 생성
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

# 선택된 변수 출력
print("Selected features by Random Forest:")
print(top_features)

# 라벨 인코딩 (이미 인코딩되어 있다고 가정)
y_train_encoded = y_train
y_test_encoded = y_test

# 다중 분류를 위한 원-핫 인코딩 (딥러닝 모델용)
y_train_onehot = to_categorical(y_train_encoded)
y_test_onehot = to_categorical(y_test_encoded)

# 모델 학습 및 평가 함수
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test, y_test)
    print(f'Test Accuracy: {accuracy*100:.2f}%')
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

# Random Forest
print("Random Forest")
evaluate_model(RandomForestClassifier(n_estimators=100, random_state=42), X_train_selected, y_train, X_test_selected, y_test)

# XGBoost
print("XGBoost")
evaluate_model(xgb.XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train)), eval_metric='mlogloss', use_label_encoder=False), X_train_selected, y_train_encoded, X_test_selected, y_test_encoded)

# LightGBM
print("LightGBM")
evaluate_model(lgb.LGBMClassifier(objective='multiclass', num_class=len(np.unique(y_train)), metric='multi_logloss'), X_train_selected, y_train, X_test_selected, y_test)

# CatBoost
print("CatBoost")
evaluate_model(CatBoostClassifier(loss_function='MultiClass', eval_metric='MultiClass', verbose=0), X_train_selected, y_train, X_test_selected, y_test)

# Decision Tree
print("Decision Tree")
evaluate_model(DecisionTreeClassifier(random_state=42), X_train_selected, y_train, X_test_selected, y_test)

# SVM
print("SVM")
evaluate_model(SVC(probability=True), X_train_selected, y_train, X_test_selected, y_test)

# 딥러닝 모델 평가 함수
def evaluate_dl_model(model, X_train, y_train, X_test, y_test, y_test_labels):
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    accuracy = np.mean(y_pred_classes == y_test_labels)
    print(f'Test Accuracy: {accuracy*100:.2f}%')
    print(classification_report(y_test_labels, y_pred_classes))
    print(confusion_matrix(y_test_labels, y_pred_classes))

# LSTM

lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(X_train_selected.shape[1], 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_lstm = X_train_selected.values.reshape((X_train_selected.shape[0], X_train_selected.shape[1], 1))
X_test_lstm = X_test_selected.values.reshape((X_test_selected.shape[0], X_test_selected.shape[1], 1))
print("LSTM")
evaluate_dl_model(lstm_model, X_train_lstm, y_train_onehot, X_test_lstm, y_test_onehot, y_test_encoded)

# 1D CNN

cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train_selected.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(50, activation='relu'))
cnn_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("1D CNN")
evaluate_dl_model(cnn_model, X_train_lstm, y_train_onehot, X_test_lstm, y_test_onehot, y_test_encoded)

# MLP

mlp_model = Sequential()
mlp_model.add(Dense(128, input_dim=X_train_selected.shape[1], activation='relu'))
mlp_model.add(Dropout(0.2))
mlp_model.add(Dense(64, activation='relu'))
mlp_model.add(Dropout(0.2))
mlp_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
mlp_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("MLP")
evaluate_dl_model(mlp_model, X_train_selected, y_train_onehot, X_test_selected, y_test_onehot, y_test_encoded)

# RNN

rnn_model = Sequential()
rnn_model.add(SimpleRNN(128, input_shape=(X_train_selected.shape[1], 1)))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("RNN")
evaluate_dl_model(rnn_model, X_train_lstm, y_train_onehot, X_test_lstm, y_test_onehot, y_test_encoded)


Feature importances sorted by Random Forest:
                          Feature  Importance
3                        Team_win    0.026732
4                    Opponent_win    0.013991
95              Team_Impact_Score    0.013231
49               Team_크로스 성공%_avg    0.012250
50   Team_vs_Opponent_크로스 성공%_avg    0.012205
..                            ...         ...
109                       Team_전남    0.000129
100                       Team_대전    0.000118
98                        Team_광주    0.000107
101                       Team_부산    0.000089
118                        상대팀_부산    0.000046

[130 rows x 2 columns]
Selected features by Random Forest:
['Team_win' 'Opponent_win' 'Team_Impact_Score' 'Team_크로스 성공%_avg'
 'Team_vs_Opponent_크로스 성공%_avg' 'Team_vs_Opponent_경합 공중 시도_avg'
 'Team_드리블 성공%_avg' 'Team_vs_Opponent_수비진영 성공%_avg' 'Team_크로스 시도_avg'
 'Team_vs_Opponent_패스 시도_avg']
Random Forest
Test Accuracy: 39.84%
              precision    recall  f1-score   support

           0       0.

c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

LSTM
Epoch 1/50
74/74 [==============================] - 4s 20ms/step - loss: 1.1348 - accuracy: 0.3627 - val_loss: 1.0896 - val_accuracy: 0.4062
Epoch 2/50
74/74 [==============================] - 1s 11ms/step - loss: 1.1048 - accuracy: 0.3619 - val_loss: 1.0921 - val_accuracy: 0.3750
Epoch 3/50
74/74 [==============================] - 1s 10ms/step - loss: 1.0990 - accuracy: 0.3623 - val_loss: 1.0944 - val_accuracy: 0.4219
Epoch 4/50
74/74 [==============================] - 1s 11ms/step - loss: 1.0920 - accuracy: 0.3860 - val_loss: 1.0845 - val_accuracy: 0.4219
Epoch 5/50
74/74 [==============================] - 1s 10ms/step - loss: 1.0882 - accuracy: 0.4038 - val_loss: 1.0895 - val_accuracy: 0.4219
Epoch 6/50
74/74 [==============================] - 1s 12ms/step - loss: 1.0820 - accuracy: 0.4114 - val_loss: 1.0854 - val_accuracy: 0.4141
Epoch 7/50
74/74 [==============================] - 1s 10ms/step - loss: 1.0800 - accuracy: 0.4199 - val_loss: 1.0832 - val_accuracy: 0.4219
Epoch 8/

c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Epoch 1/50
74/74 [==============================] - 2s 7ms/step - loss: 6.7444 - accuracy: 0.3331 - val_loss: 2.0813 - val_accuracy: 0.3828
Epoch 2/50
74/74 [==============================] - 0s 3ms/step - loss: 3.4049 - accuracy: 0.3305 - val_loss: 1.5505 - val_accuracy: 0.3672
Epoch 3/50
74/74 [==============================] - 0s 3ms/step - loss: 2.4131 - accuracy: 0.3386 - val_loss: 1.0926 - val_accuracy: 0.4219
Epoch 4/50
74/74 [==============================] - 0s 3ms/step - loss: 1.9507 - accuracy: 0.3373 - val_loss: 1.0919 - val_accuracy: 0.4141
Epoch 5/50
74/74 [==============================] - 0s 4ms/step - loss: 1.7011 - accuracy: 0.3631 - val_loss: 1.1443 - val_accuracy: 0.4297
Epoch 6/50
74/74 [==============================] - 0s 4ms/step - loss: 1.5086 - accuracy: 0.3665 - val_loss: 1.2394 - val_accuracy: 0.2812
Epoch 7/50
74/74 [==============================] - 0s 3ms/step - loss: 1.4430 - accuracy: 0.3513 - val_loss: 1.1576 - val_accuracy: 0.2891
Epoch 8/50
74/74 [==

c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

74/74 [==============================] - 2s 10ms/step - loss: 1.1582 - accuracy: 0.3551 - val_loss: 1.0887 - val_accuracy: 0.3672
Epoch 2/50
74/74 [==============================] - 0s 5ms/step - loss: 1.1155 - accuracy: 0.3555 - val_loss: 1.0944 - val_accuracy: 0.3672
Epoch 3/50
74/74 [==============================] - 0s 5ms/step - loss: 1.0995 - accuracy: 0.3407 - val_loss: 1.0900 - val_accuracy: 0.3672
Epoch 4/50
74/74 [==============================] - 0s 5ms/step - loss: 1.0979 - accuracy: 0.3445 - val_loss: 1.0910 - val_accuracy: 0.3672
Epoch 5/50
74/74 [==============================] - 0s 6ms/step - loss: 1.0960 - accuracy: 0.3547 - val_loss: 1.0897 - val_accuracy: 0.3672
Epoch 6/50
74/74 [==============================] - 0s 5ms/step - loss: 1.0970 - accuracy: 0.3525 - val_loss: 1.0912 - val_accuracy: 0.3672
Epoch 7/50
74/74 [==============================] - 0s 5ms/step - loss: 1.0973 - accuracy: 0.3534 - val_loss: 1.0913 - val_accuracy: 0.3672
Epoch 8/50
74/74 [============

c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SimpleRNN
from tensorflow.keras.utils import to_categorical

# 데이터 전처리
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Lasso 회귀를 사용하여 변수 선택
lasso = LassoCV(cv=5, random_state=42).fit(X_train_scaled, y_train)
importance = np.abs(lasso.coef_)
selected_features = np.where(importance > 0)[0]
X_train_selected = X_train_scaled[:, selected_features]
X_test_selected = X_test_scaled[:, selected_features]

# 선택된 변수 출력
selected_feature_names = X_train.columns[selected_features]
print("Selected features by Lasso:")
print(selected_feature_names)

# 라벨 인코딩 (이미 인코딩되어 있다고 가정)
y_train_encoded = y_train
y_test_encoded = y_test

# 다중 분류를 위한 원-핫 인코딩 (딥러닝 모델용)
y_train_onehot = to_categorical(y_train_encoded)
y_test_onehot = to_categorical(y_test_encoded)

# 모델 학습 및 평가 함수
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test, y_test)
    print(f'Test Accuracy: {accuracy*100:.2f}%')
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

# Random Forest
print("Random Forest")
evaluate_model(RandomForestClassifier(n_estimators=100, random_state=42), X_train_selected, y_train, X_test_selected, y_test)

# XGBoost
print("XGBoost")
evaluate_model(xgb.XGBClassifier(objective='multi:softmax', num_class=len(np.unique(y_train)), eval_metric='mlogloss', use_label_encoder=False), X_train_selected, y_train_encoded, X_test_selected, y_test_encoded)

# LightGBM
print("LightGBM")
evaluate_model(lgb.LGBMClassifier(objective='multiclass', num_class=len(np.unique(y_train)), metric='multi_logloss'), X_train_selected, y_train, X_test_selected, y_test)

# CatBoost
print("CatBoost")
evaluate_model(CatBoostClassifier(loss_function='MultiClass', eval_metric='MultiClass', verbose=0), X_train_selected, y_train, X_test_selected, y_test)

# Decision Tree
print("Decision Tree")
evaluate_model(DecisionTreeClassifier(random_state=42), X_train_selected, y_train, X_test_selected, y_test)

# SVM
print("SVM")
evaluate_model(SVC(probability=True), X_train_selected, y_train, X_test_selected, y_test)

# 딥러닝 모델 평가 함수
def evaluate_dl_model(model, X_train, y_train, X_test, y_test, y_test_labels):
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    accuracy = np.mean(y_pred_classes == y_test_labels)
    print(f'Test Accuracy: {accuracy*100:.2f}%')
    print(classification_report(y_test_labels, y_pred_classes))
    print(confusion_matrix(y_test_labels, y_pred_classes))

# LSTM

lstm_model = Sequential()
lstm_model.add(LSTM(128, input_shape=(X_train_selected.shape[1], 1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
X_train_lstm = X_train_selected.reshape((X_train_selected.shape[0], X_train_selected.shape[1], 1))
X_test_lstm = X_test_selected.reshape((X_test_selected.shape[0], X_test_selected.shape[1], 1))
print("LSTM")
evaluate_dl_model(lstm_model, X_train_lstm, y_train_onehot, X_test_lstm, y_test_onehot, y_test_encoded)

# 1D CNN
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train_selected.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.2))
cnn_model.add(Flatten())
cnn_model.add(Dense(50, activation='relu'))
cnn_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("1D CNN")
evaluate_dl_model(cnn_model, X_train_lstm, y_train_onehot, X_test_lstm, y_test_onehot, y_test_encoded)

# MLP
mlp_model = Sequential()
mlp_model.add(Dense(128, input_dim=X_train_selected.shape[1], activation='relu'))
mlp_model.add(Dropout(0.2))
mlp_model.add(Dense(64, activation='relu'))
mlp_model.add(Dropout(0.2))
mlp_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
mlp_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("MLP")
evaluate_dl_model(mlp_model, X_train_selected, y_train_onehot, X_test_selected, y_test_onehot, y_test_encoded)

# RNN
rnn_model = Sequential()
rnn_model.add(SimpleRNN(128, input_shape=(X_train_selected.shape[1], 1)))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(len(np.unique(y_train)), activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("RNN")
evaluate_dl_model(rnn_model, X_train_lstm, y_train_onehot, X_test_lstm, y_test_onehot, y_test_encoded)


c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5100909436337133, tolerance: 0.134693591101695
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.540126180553898, tolerance: 0.134693591101695
  model = cd_fast.enet_coordinate_descent_gram(
c:\Users\jungi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.989221077713978, tolerance: 0.134693591101695
  model = cd_fast.enet_coordinate_descent_gram(
c:\

Selected features by Lasso:
Index(['H/A', 'Draw', 'Team_win', 'Opponent_win', 'Team_vs_Opponent_슈팅_avg',
       'Team_vs_Opponent_유효 슈팅_avg', 'Team_vs_Opponent_키패스_avg',
       'Team_크로스 성공_avg', 'Team_vs_Opponent_실점_avg', 'Team_Impact_Score',
       'Team_부산', 'Team_수원', 'Team_울산', 'Team_인천', 'Team_전남', 'Team_전북',
       'Team_제주', 'Team_포항', '상대팀_대구', '상대팀_부산', '상대팀_성남', '상대팀_수원', '상대팀_울산',
       '상대팀_전남', '상대팀_전북', '상대팀_포항'],
      dtype='object')
Random Forest
Test Accuracy: 37.50%
              precision    recall  f1-score   support

           0       0.40      0.43      0.41        47
           1       0.25      0.21      0.23        34
           2       0.42      0.45      0.43        47

    accuracy                           0.38       128
   macro avg       0.36      0.36      0.36       128
weighted avg       0.37      0.38      0.37       128

[[20 10 17]
 [15  7 12]
 [15 11 21]]
XGBoost
Test Accuracy: 34.38%
              precision    recall  f1-score   support

     